In [1]:
import pandas as pd
import numpy as np

In [2]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

In [3]:
from sqlalchemy import BigInteger, Boolean, CheckConstraint, Column, DateTime, Float, ForeignKey, Integer, SmallInteger, String, Text, UniqueConstraint
from sqlalchemy.orm import relationship

In [4]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'postgresql+psycopg2://postgres@localhost/dev_logware3'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)
conn = db.engine.connect().connection

In [5]:
class Annotation(db.Model):
    __tablename__ = 'annotations'

    annotation_guid = db.Column(db.String(32), primary_key=True)
    reading_guid = db.Column(db.ForeignKey('readings.reading_guid'), nullable=False)
    annotation = db.Column(db.Text)

    reading = db.relationship('Reading', primaryjoin='Annotation.reading_guid == Reading.reading_guid', backref='annotations')


class Asset(db.Model):
    __tablename__ = 'assets'
    __table_args__ = (
        db.CheckConstraint("(model)::text <> ''::text"),
        db.CheckConstraint("(serial)::text <> ''::text"),
        db.UniqueConstraint('model', 'serial')
    )

    asset_guid = db.Column(db.String(32), primary_key=True)
    asset_type = db.Column(db.SmallInteger, nullable=False)
    model = db.Column(db.String(32), nullable=False)
    serial = db.Column(db.String(32), nullable=False)
    active = db.Column(db.Boolean)
    deleted = db.Column(db.Boolean)
    asset_password = db.Column(db.String(20))
    notes = db.Column(db.Text)


class LicenseInUse(db.Model):
    __tablename__ = 'license_in_use'

    license_in_use_guid = db.Column(db.String(32), primary_key=True)
    computer_name = db.Column(db.Text, nullable=False)
    user_guid = db.Column(db.ForeignKey('users.user_guid'), nullable=False)
    license_guid = db.Column(db.ForeignKey('licenses.license_guid'), nullable=False)
    time_stamp = db.Column(db.DateTime, nullable=False)

    license = db.relationship('License', primaryjoin='LicenseInUse.license_guid == License.license_guid', backref='license_in_uses')
    user = db.relationship('User', primaryjoin='LicenseInUse.user_guid == User.user_guid', backref='license_in_uses')


class License(db.Model):
    __tablename__ = 'licenses'

    license_guid = db.Column(db.String(32), primary_key=True)
    license_type = db.Column(db.SmallInteger, nullable=False)
    license_serial = db.Column(db.String(20))
    version = db.Column(db.String(20))
    date_applied = db.Column(db.DateTime, nullable=False)
    logins_remaining = db.Column(db.Integer)
    license_id = db.Column(db.Text, nullable=False)
    deleted = db.Column(db.Boolean)


class Location(db.Model):
    __tablename__ = 'locations'
    __table_args__ = (
        db.CheckConstraint("(location_name)::text <> ''::text"),
    )

    location_guid = db.Column(db.String(32), primary_key=True)
    location_name = db.Column(db.String(20), nullable=False, unique=True)
    active = db.Column(db.Boolean)
    deleted = db.Column(db.Boolean)
    notes = db.Column(db.Text)


class LogSession(db.Model):
    __tablename__ = 'log_sessions'

    log_session_guid = db.Column(db.String(32), primary_key=True)
    session_start = db.Column(db.DateTime, nullable=False)
    session_end = db.Column(db.DateTime)
    logging_interval = db.Column(db.Integer, nullable=False)
    logger_guid = db.Column(db.ForeignKey('assets.asset_guid'), nullable=False)
    user_guid = db.Column(db.ForeignKey('users.user_guid'), nullable=False)
    session_type = db.Column(db.SmallInteger, nullable=False)
    computer_name = db.Column(db.Text, nullable=False)

    asset = db.relationship('Asset', primaryjoin='LogSession.logger_guid == Asset.asset_guid', backref='log_sessions')
    user = db.relationship('User', primaryjoin='LogSession.user_guid == User.user_guid', backref='log_sessions')


class Reading(db.Model):
    __tablename__ = 'readings'

    reading_guid = db.Column(db.String(32), primary_key=True)
    reading = db.Column(db.Float(53), nullable=False)
    reading_type = db.Column(db.SmallInteger, nullable=False)
    time_stamp = db.Column(db.DateTime, nullable=False)
    log_session_guid = db.Column(db.ForeignKey('log_sessions.log_session_guid'), nullable=False)
    sensor_guid = db.Column(db.ForeignKey('assets.asset_guid'), nullable=False)
    location_guid = db.Column(db.ForeignKey('locations.location_guid'), nullable=False)
    channel = db.Column(db.SmallInteger, nullable=False)
    max_alarm = db.Column(db.Boolean)
    max_alarm_value = db.Column(db.Float(53))
    min_alarm = db.Column(db.Boolean)
    min_alarm_value = db.Column(db.Float(53))
    compromised = db.Column(db.Boolean)

    location = db.relationship('Location', primaryjoin='Reading.location_guid == Location.location_guid', backref='readings')
    log_session = db.relationship('LogSession', primaryjoin='Reading.log_session_guid == LogSession.log_session_guid', backref='readings')
    asset = db.relationship('Asset', primaryjoin='Reading.sensor_guid == Asset.asset_guid', backref='readings')


class SensorParameter(db.Model):
    __tablename__ = 'sensor_parameters'

    log_session_guid = db.Column(db.ForeignKey('log_sessions.log_session_guid'), primary_key=True, nullable=False)
    channel = db.Column(db.SmallInteger, primary_key=True, nullable=False)
    parameter_name = db.Column(db.String(128), primary_key=True, nullable=False)
    parameter_value = db.Column(db.String(128), nullable=False)

    log_session = db.relationship('LogSession', primaryjoin='SensorParameter.log_session_guid == LogSession.log_session_guid', backref='sensor_parameters')


class User(db.Model):
    __tablename__ = 'users'
    __table_args__ = (
        db.CheckConstraint("(login_name)::text <> ''::text"),
    )

    user_guid = db.Column(db.String(32), primary_key=True)
    login_name = db.Column(db.String(32), nullable=False, unique=True)
    first_name = db.Column(db.String(64))
    last_name = db.Column(db.String(64))
    user_password = db.Column(db.String(64))
    user_group = db.Column(db.SmallInteger)
    permissions = db.Column(db.BigInteger)
    active = db.Column(db.Boolean)
    deleted = db.Column(db.Boolean)
    change = db.Column(db.Boolean)
    notes = db.Column(db.Text)


class Version(db.Model):
    __tablename__ = 'versions'

    db_version = db.Column(db.String(20), primary_key=True)
    client_version = db.Column(db.String(20))

In [6]:
records = d    SELECT readings.reading_guid, readings.reading , readings.reading_type , readings.time_stamp , locations.location_name , readings.sensor_guid , readings.location_guid AS readings_location_guid, readings.channel AS readings_channel, readings.max_alarm AS readings_max_alarm, readings.max_alarm_value AS readings_max_alarm_value, readings.min_alarm AS readings_min_alarm, readings.min_alarm_value AS readings_min_alarm_value, readings.compromised AS readings_compromised 
b.session.query(Reading).join(Reading.location).filter_by(location_name='ONSITE1').filter(Reading.time_stamp.between('2017-03-26', '2017-03-28'))

SyntaxError: invalid syntax (<ipython-input-6-53a3ce4be2d7>, line 1)

In [7]:
criteria = {"location_name_1": 'ONSITE1', "time_stamp_1":'2017-01-26', "time_stamp_2":'2017-03-28'}

In [8]:
df = pd.read_sql(('''
    SELECT readings.reading_guid, readings.reading , readings.reading_type , readings.time_stamp , locations.location_name , readings.sensor_guid , readings.location_guid AS readings_location_guid, readings.channel AS readings_channel, readings.max_alarm AS readings_max_alarm, readings.max_alarm_value AS readings_max_alarm_value, readings.min_alarm AS readings_min_alarm, readings.min_alarm_value AS readings_min_alarm_value, readings.compromised AS readings_compromised 
FROM readings JOIN locations ON readings.location_guid = locations.location_guid 
WHERE locations.location_name = %(location_name_1)s AND readings.time_stamp BETWEEN %(time_stamp_1)s AND %(time_stamp_2)s'''),
                conn, params=criteria)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2764 entries, 0 to 2763
Data columns (total 13 columns):
reading_guid                2764 non-null object
reading                     2764 non-null float64
reading_type                2764 non-null int64
time_stamp                  2764 non-null datetime64[ns]
location_name               2764 non-null object
sensor_guid                 2764 non-null object
readings_location_guid      2764 non-null object
readings_channel            2764 non-null int64
readings_max_alarm          2764 non-null bool
readings_max_alarm_value    1888 non-null float64
readings_min_alarm          2764 non-null bool
readings_min_alarm_value    1888 non-null float64
readings_compromised        2764 non-null bool
dtypes: bool(3), datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 224.1+ KB


In [9]:
df

,reading_guid,reading,reading_type,time_stamp,location_name,sensor_guid,readings_location_guid,readings_channel,readings_max_alarm,readings_max_alarm_value,readings_min_alarm,readings_min_alarm_value,readings_compromised
0,79003BACCFCD47C3BFAFD8609267FFC7,21.220000,0,2017-01-31 17:18:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
1,B9CCF105B513480DB2FE7FF6127E4867,31.500000,1,2017-01-31 17:18:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
2,CDA88C7DCA724E3C805B7E59E9FE3683,22.120000,0,2017-01-31 17:20:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
3,BADFB667C0F445A7A846AAA948814F3C,32.200000,1,2017-01-31 17:20:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
4,BA9C5056F9604908B3D2D6DF516490E4,22.100000,0,2017-01-31 17:22:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
5,D2AF5F276B8345EFAA0253C6BAD4F131,30.200000,1,2017-01-31 17:22:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
6,B518421AF8CB4526A86F49C25041B076,22.000000,0,2017-01-31 17:24:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
7,51AB4ED120A0487C991811C295302418,29.600000,1,2017-01-31 17:24:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
8,E56D99F16C3D469F83D0B2FAD0FA9802,21.890000,0,2017-01-31 17:26:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
9,D4CA983B7D6E457488471D54D879A5EC,29.300000,1,2017-01-31 17:26:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False


In [10]:
def celsius_to_fahr(temp_celsius):
    """Convert Fahrenheit to Celsius
    
    Return Celsius conversion of input"""
    temp_fahr = (temp_celsius * 1.8) + 32
    return temp_fahr

In [11]:
# convert temps to fahrenheit
df.loc[df['reading_type'] == 0, 'reading'] = df.reading.apply(celsius_to_fahr)
df

,reading_guid,reading,reading_type,time_stamp,location_name,sensor_guid,readings_location_guid,readings_channel,readings_max_alarm,readings_max_alarm_value,readings_min_alarm,readings_min_alarm_value,readings_compromised
0,79003BACCFCD47C3BFAFD8609267FFC7,70.196,0,2017-01-31 17:18:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
1,B9CCF105B513480DB2FE7FF6127E4867,31.500,1,2017-01-31 17:18:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
2,CDA88C7DCA724E3C805B7E59E9FE3683,71.816,0,2017-01-31 17:20:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
3,BADFB667C0F445A7A846AAA948814F3C,32.200,1,2017-01-31 17:20:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
4,BA9C5056F9604908B3D2D6DF516490E4,71.780,0,2017-01-31 17:22:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
5,D2AF5F276B8345EFAA0253C6BAD4F131,30.200,1,2017-01-31 17:22:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
6,B518421AF8CB4526A86F49C25041B076,71.600,0,2017-01-31 17:24:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
7,51AB4ED120A0487C991811C295302418,29.600,1,2017-01-31 17:24:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
8,E56D99F16C3D469F83D0B2FAD0FA9802,71.402,0,2017-01-31 17:26:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False
9,D4CA983B7D6E457488471D54D879A5EC,29.300,1,2017-01-31 17:26:00,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False


In [12]:
locs = [loc for loc, in db.session.query(Location.location_name)]

In [13]:
summary = pd.DataFrame(index=None, columns=['LOCATION', 'SPECIFICATION', 'START_DATE', 'END_DATE', 'FIRST_POINT_RECORDED', 'LAST_POINT_RECORDED', 'TOTAL_HOURS_EVALUATED', 'TOTAL_HOURS_RECORDED', 'TOTAL_HOURS_OUT', 'PERCENT_OUT', 'HOURS_TEMP_HIGH', 'HOURS_TEMP_LOW', 'HOURS_RH_HIGH', 'HOURS_RH_LOW', 'HOURS_OVERLAP', 'HOURS_NO_DATA', 'INT_GREATER_THAN_15', 'HRS_DOWN_FOR_MAINT', 'DUPE_RECORDS'])
summary

,LOCATION,SPECIFICATION,START_DATE,END_DATE,FIRST_POINT_RECORDED,LAST_POINT_RECORDED,TOTAL_HOURS_EVALUATED,TOTAL_HOURS_RECORDED,TOTAL_HOURS_OUT,PERCENT_OUT,HOURS_TEMP_HIGH,HOURS_TEMP_LOW,HOURS_RH_HIGH,HOURS_RH_LOW,HOURS_OVERLAP,HOURS_NO_DATA,INT_GREATER_THAN_15,HRS_DOWN_FOR_MAINT,DUPE_RECORDS


In [14]:
df.location_name.unique()

array(['ONSITE1'], dtype=object)

In [15]:
summary.LOCATION = df.location_name.unique()
summary

,LOCATION,SPECIFICATION,START_DATE,END_DATE,FIRST_POINT_RECORDED,LAST_POINT_RECORDED,TOTAL_HOURS_EVALUATED,TOTAL_HOURS_RECORDED,TOTAL_HOURS_OUT,PERCENT_OUT,HOURS_TEMP_HIGH,HOURS_TEMP_LOW,HOURS_RH_HIGH,HOURS_RH_LOW,HOURS_OVERLAP,HOURS_NO_DATA,INT_GREATER_THAN_15,HRS_DOWN_FOR_MAINT,DUPE_RECORDS
0,ONSITE1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
temps = df[df['reading_type']==0]
temps.dtypes

reading_guid                        object
reading                            float64
reading_type                         int64
time_stamp                  datetime64[ns]
location_name                       object
sensor_guid                         object
readings_location_guid              object
readings_channel                     int64
readings_max_alarm                    bool
readings_max_alarm_value           float64
readings_min_alarm                    bool
readings_min_alarm_value           float64
readings_compromised                  bool
dtype: object

In [17]:
temps = temps.set_index('time_stamp')

In [18]:
temps['duration'] = temps.index.to_series().diff().dt.seconds.div(60, fill_value=0)
temps.describe()

,reading,reading_type,readings_channel,readings_max_alarm_value,readings_min_alarm_value,duration
count,1382.000000,1382.0,1382.0,9.440000e+02,9.440000e+02,1382.000000
mean,69.912776,0.0,1.0,2.611111e+01,1.944444e+01,4.457622
std,1.701760,0.0,0.0,7.109194e-15,7.109194e-15,13.596447
min,68.530000,0.0,1.0,2.611111e+01,1.944444e+01,0.000000
25%,68.760000,0.0,1.0,2.611111e+01,1.944444e+01,2.000000
50%,69.025000,0.0,1.0,2.611111e+01,1.944444e+01,5.000000
75%,70.610000,0.0,1.0,2.611111e+01,1.944444e+01,5.000000
max,75.720000,0.0,1.0,2.611111e+01,1.944444e+01,505.416667


In [19]:
temp_hi = temps[temps.reading > 72]
temp_low = temps[temps.reading < 69]
temp_low.describe()

,reading,reading_type,readings_channel,readings_max_alarm_value,readings_min_alarm_value,duration
count,686.000000,686.0,686.0,6.860000e+02,6.860000e+02,686.000000
mean,68.756458,0.0,1.0,2.611111e+01,1.944444e+01,5.000073
std,0.097334,0.0,0.0,3.555306e-15,7.110612e-15,0.004635
min,68.530000,0.0,1.0,2.611111e+01,1.944444e+01,4.983333
25%,68.690000,0.0,1.0,2.611111e+01,1.944444e+01,5.000000
50%,68.760000,0.0,1.0,2.611111e+01,1.944444e+01,5.000000
75%,68.820000,0.0,1.0,2.611111e+01,1.944444e+01,5.000000
max,69.000000,0.0,1.0,2.611111e+01,1.944444e+01,5.016667


In [20]:
t_hr_hi = temp_hi.duration.sum(axis=0) / 60
t_hr_hi

11.916944444444445

In [21]:
t_hr_low = temp_low.duration.sum(axis=0) / 60
t_hr_low

57.167500000000004

In [22]:
t_gap = temps[temps.duration > 15]
t_gap_hrs = t_gap.duration.sum(axis=0) / 60
t_gap

,reading_guid,reading,reading_type,location_name,sensor_guid,readings_location_guid,readings_channel,readings_max_alarm,readings_max_alarm_value,readings_min_alarm,readings_min_alarm_value,readings_compromised,duration
time_stamp,,,,,,,,,,,,,
2017-01-31 21:20:00,11685A2F4192469D9485DD7D99CA41D5,70.556,0,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False,16.000000
2017-01-31 23:04:00,35039C3117AD435E9DEA3F94CF20293F,70.844,0,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False,40.000000
2017-03-24 17:23:25,CDA90AAE6F5E46819C58E879E3A793F0,71.030,0,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,True,26.111111,True,19.444444,False,505.416667


In [23]:
t_gap_hrs

9.3569444444444461

In [24]:
# df.loc[df['line_race'] == 0, 'rating'] = 0
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'START_DATE'] = pd.to_datetime(criteria.get('time_stamp_1'))
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'END_DATE'] = pd.to_datetime(criteria.get('time_stamp_2'))
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'FIRST_POINT_RECORDED'] = df.time_stamp.min()
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'LAST_POINT_RECORDED'] = df.time_stamp.max()
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'TOTAL_HOURS_EVALUATED'] = (summary.END_DATE - summary.START_DATE).astype('timedelta64[s]') / 3600.0
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'TOTAL_HOURS_RECORDED'] = (summary.LAST_POINT_RECORDED - summary.FIRST_POINT_RECORDED).astype('timedelta64[s]') / 3600.0
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'HOURS_TEMP_HIGH'] = temp_hi.duration.sum(axis=0) / 60
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'HOURS_TEMP_LOW'] = temp_low.duration.sum(axis=0) / 60
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'HOURS_RH_HIGH'] = rh_hi.duration.sum(axis=0) / 60
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'HOURS_RH_LOW'] = rh_low.duration.sum(axis=0) / 60
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'TOTAL_HOURS_RECORDED'] = (((summary.LAST_POINT_RECORDED - summary.FIRST_POINT_RECORDED).astype('timedelta64[s]') / 3600.0) -(t_gap_hrs / 60))
summary

NameError: name 'rh_hi' is not defined

In [25]:
rh = df[df['reading_type']==1]
rh.dtypes

reading_guid                        object
reading                            float64
reading_type                         int64
time_stamp                  datetime64[ns]
location_name                       object
sensor_guid                         object
readings_location_guid              object
readings_channel                     int64
readings_max_alarm                    bool
readings_max_alarm_value           float64
readings_min_alarm                    bool
readings_min_alarm_value           float64
readings_compromised                  bool
dtype: object

In [26]:
rh = rh.set_index('time_stamp')

In [27]:
rh['duration'] = rh.index.to_series().diff().dt.seconds.div(60, fill_value=0)
rh.describe()

,reading,reading_type,readings_channel,readings_max_alarm_value,readings_min_alarm_value,duration
count,1382.000000,1382.0,1382.0,944.0,944.0,1382.000000
mean,27.414761,1.0,1.0,50.0,20.0,4.457622
std,2.687415,0.0,0.0,0.0,0.0,13.596447
min,20.700000,1.0,1.0,50.0,20.0,0.000000
25%,25.400000,1.0,1.0,50.0,20.0,2.000000
50%,28.200000,1.0,1.0,50.0,20.0,5.000000
75%,29.500000,1.0,1.0,50.0,20.0,5.000000
max,32.200000,1.0,1.0,50.0,20.0,505.416667


In [28]:
rh_hi = rh[rh.reading > 29.5]
rh_low = rh[rh.reading < 25.5]
rh_low.describe()

,reading,reading_type,readings_channel,readings_max_alarm_value,readings_min_alarm_value,duration
count,361.000000,361.0,361.0,361.0,361.0,361.000000
mean,23.571468,1.0,1.0,50.0,20.0,5.000092
std,1.132029,0.0,0.0,0.0,0.0,0.005957
min,20.700000,1.0,1.0,50.0,20.0,4.966667
25%,22.900000,1.0,1.0,50.0,20.0,5.000000
50%,23.700000,1.0,1.0,50.0,20.0,5.000000
75%,24.500000,1.0,1.0,50.0,20.0,5.000000
max,25.400000,1.0,1.0,50.0,20.0,5.016667


In [29]:
rh_hr_hi = rh_hi.duration.sum(axis=0) / 60
rh_hr_hi

11.533333333333333

In [30]:
rh_hr_low = rh_low.duration.sum(axis=0) / 60
rh_hr_low

30.08388888888889

In [31]:
rh_gap = rh[rh.duration > 15]
rh_gap_hrs = rh_gap.duration.sum(axis=0)
rh_gap

,reading_guid,reading,reading_type,location_name,sensor_guid,readings_location_guid,readings_channel,readings_max_alarm,readings_max_alarm_value,readings_min_alarm,readings_min_alarm_value,readings_compromised,duration
time_stamp,,,,,,,,,,,,,
2017-01-31 21:20:00,814C2C1E50134569832A7EAF2886F06F,30.4,1,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False,16.000000
2017-01-31 23:04:00,E69520CD72354F208F5D39E41F6759CD,30.9,1,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,False,NaN,False,NaN,False,40.000000
2017-03-24 17:23:25,8101FCCC420B45DCB0CE915DAA9F32B7,27.8,1,ONSITE1,E8E46C0B8F154CC185E9099128CC8BBD,16B4E541EC4546E682F040039B8CE058,1,True,50.0,True,20.0,False,505.416667


In [32]:
rh_gap_hrs

561.41666666666674

In [33]:
a = ((summary.LAST_POINT_RECORDED - summary.FIRST_POINT_RECORDED).astype('timedelta64[s]') / 3600.0)

In [34]:
a - t_gap_hrs

0    1317.316944
dtype: float64

In [35]:
pd.concat([t_gap, rh_gap], keys=['reading_type', 'time_stamp'])

reading_guid  reading  \
             time_stamp                                                       
reading_type 2017-01-31 21:20:00  11685A2F4192469D9485DD7D99CA41D5   70.556   
             2017-01-31 23:04:00  35039C3117AD435E9DEA3F94CF20293F   70.844   
             2017-03-24 17:23:25  CDA90AAE6F5E46819C58E879E3A793F0   71.030   
time_stamp   2017-01-31 21:20:00  814C2C1E50134569832A7EAF2886F06F   30.400   
             2017-01-31 23:04:00  E69520CD72354F208F5D39E41F6759CD   30.900   
             2017-03-24 17:23:25  8101FCCC420B45DCB0CE915DAA9F32B7   27.800   

                                  reading_type location_name  \
             time_stamp                                        
reading_type 2017-01-31 21:20:00             0       ONSITE1   
             2017-01-31 23:04:00             0       ONSITE1   
             2017-03-24 17:23:25             0       ONSITE1   
time_stamp   2017-01-31 21:20:00             1       ONSITE1   
             2017-01-31 23:04:00             1       ONSITE1   
             2017-03-24 17:23:25             1       ONSITE1   

                                                       sensor_guid  \
             time_stamp                                              
reading_type 2017-01-31 21:20:00  E8E46C0B8F154CC185E9099128CC8BBD   
             2017-01-31 23:04:00  E8E46C0B8F154CC185E9099128CC8BBD   
             2017-03-24 17:23:25  E8E46C0B8F154CC185E9099128CC8BBD   
time_stamp   2017-01-31 21:20:00  E8E46C0B8F154CC185E9099128CC8BBD   
             2017-01-31 23:04:00  E8E46C0B8F154CC185E9099128CC8BBD   
             2017-03-24 17:23:25  E8E46C0B8F154CC185E9099128CC8BBD   

                                            readings_location_guid  \
             time_stamp                                              
reading_type 2017-01-31 21:20:00  16B4E541EC4546E682F040039B8CE058   
             2017-01-31 23:04:00  16B4E541EC4546E682F040039B8CE058   
             2017-03-24 17:23:25  16B4E541EC4546E682F040039B8CE058   
time_stamp   2017-01-31 21:20:00  16B4E541EC4546E682F040039B8CE058   
             2017-01-31 23:04:00  16B4E541EC4546E682F040039B8CE058   
             2017-03-24 17:23:25  16B4E541EC4546E682F040039B8CE058   

                                  readings_channel readings_max_alarm  \
             time_stamp                                                 
reading_type 2017-01-31 21:20:00                 1              False   
             2017-01-31 23:04:00                 1              False   
             2017-03-24 17:23:25                 1               True   
time_stamp   2017-01-31 21:20:00                 1              False   
             2017-01-31 23:04:00                 1              False   
             2017-03-24 17:23:25                 1               True   

                                  readings_max_alarm_value readings_min_alarm  \
             time_stamp                                                         
reading_type 2017-01-31 21:20:00                       NaN              False   
             2017-01-31 23:04:00                       NaN              False   
             2017-03-24 17:23:25                 26.111111               True   
time_stamp   2017-01-31 21:20:00                       NaN              False   
             2017-01-31 23:04:00                       NaN              False   
             2017-03-24 17:23:25                 50.000000               True   

                                  readings_min_alarm_value  \
             time_stamp                                      
reading_type 2017-01-31 21:20:00                       NaN   
             2017-01-31 23:04:00                       NaN   
             2017-03-24 17:23:25                 19.444444   
time_stamp   2017-01-31 21:20:00                       NaN   
             2017-01-31 23:04:00                       NaN   
             2017-03-24 17:23:25                 20.000000   

                                 readings_compromised  

In [36]:
len(pd.merge(t_gap, rh_gap, left_index=True, right_index=True))

3

In [37]:
# df.loc[df['line_race'] == 0, 'rating'] = 0
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'START_DATE'] = pd.to_datetime(criteria.get('time_stamp_1'))
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'END_DATE'] = pd.to_datetime(criteria.get('time_stamp_2'))
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'FIRST_POINT_RECORDED'] = df.time_stamp.min()
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'LAST_POINT_RECORDED'] = df.time_stamp.max()
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'TOTAL_HOURS_EVALUATED'] = (summary.END_DATE - summary.START_DATE).astype('timedelta64[s]') / 3600.0
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'HOURS_TEMP_HIGH'] = temp_hi.duration.sum(axis=0) / 60
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'HOURS_TEMP_LOW'] = temp_low.duration.sum(axis=0) / 60
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'TOTAL_HOURS_RECORDED'] = ((summary.LAST_POINT_RECORDED - summary.FIRST_POINT_RECORDED).astype('timedelta64[s]') / 3600.0) - t_gap_hrs
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'HOURS_NO_DATA'] = t_gap_hrs
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'INT_GREATER_THAN_15'] = len(pd.merge(t_gap, rh_gap, left_index=True, right_index=True))
summary.loc[summary.LOCATION == criteria.get('location_name_1'), 'TOTAL_HOURS_OUT'] = summary[['HOURS_TEMP_HIGH', 'HOURS_TEMP_LOW', 'HOURS_RH_HIGH', 'HOURS_RH_LOW']].sum(axis=1)

summary

,LOCATION,SPECIFICATION,START_DATE,END_DATE,FIRST_POINT_RECORDED,LAST_POINT_RECORDED,TOTAL_HOURS_EVALUATED,TOTAL_HOURS_RECORDED,TOTAL_HOURS_OUT,PERCENT_OUT,HOURS_TEMP_HIGH,HOURS_TEMP_LOW,HOURS_RH_HIGH,HOURS_RH_LOW,HOURS_OVERLAP,HOURS_NO_DATA,INT_GREATER_THAN_15,HRS_DOWN_FOR_MAINT,DUPE_RECORDS
0,ONSITE1,NaN,2017-01-26 00:00:00,2017-03-28 00:00:00,2017-01-31 17:18:00,2017-03-27 23:58:26,1464,1317.32,69.0844,NaN,11.9169,57.1675,NaN,NaN,NaN,9.35694,3,NaN,NaN


In [38]:
criteria.get('location_name_1')

'ONSITE1'